## Baseball Prediction: 5b - Adding Starting Pitching Features
Now that we have raw game-level data for each pitcher, we can derive features based on the starting pitchers to help our prediction model for individual games.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)



## Overall Plan of Attack
- For each starting pitcher we will load their raw data, create features for each game based on their previous performance, and then save the dataframe in a dictionary structure for easy lookup
- Then we can iterate through our game-level dataframe, add in the features for each starting pitcher, and use those to improve our model


In [ ]:
df_gooden = pd.read_csv('data/pitcher_data/pitching_data_goodd001.csv')

In [ ]:
df_gooden.info()

In [ ]:
df_gooden.head(20)

In [ ]:
df_gooden['IP_real'] = df_gooden['IP_real'] = (df_gooden.IP - (df_gooden.IP % 1)) + (df_gooden.IP % 1) * (10/3)
df_gooden.head(10)

In [ ]:
df_gooden.columns

### Generating Features for a Starting Pitcher
- Want to creature features based on a lookback across $n$ games for each pitcher
- Need to aggregate statistics like innings pitched (IP), runs or earned runs given up, hits and walks given up, strikeouts, and so on
- We will also need to decide how to handle pitchers early in their career when they have relatively few games on which to base their performance.

In [ ]:
def roll_column(df, col, winsize):
    # do the standard Pandas rolling calc
    t_col = df[col].rolling(winsize, closed='left').sum().to_numpy()
    
    # for the early columns, just do a rolling sum from the beginning
    t_col[:winsize] = np.concatenate(([0],df[col].iloc[:(winsize)].cumsum().to_numpy()[:-1]))

    return(t_col)

In [ ]:
def load_and_process_pitch_df(p_id, filepath=''):
    fname = filepath+'pitching_data_'+p_id+'.csv'
    pitch_df = pd.read_csv(fname)
    
    # Convert date, fix dblhead_num to be 0,1,2
    pitch_df['Date'] = (pd.to_datetime(pitch_df.Date).astype(str).str.replace('-','')).astype(int)
    pitch_df.dblhead_num.fillna(0, inplace=True)
    pitch_df['dblhead_num'] = pitch_df['dblhead_num'].astype(int)
    
    # Convert IP to proper mathematical format
    pitch_df['IP_real'] = (pitch_df.IP - (pitch_df.IP % 1)) + (pitch_df.IP % 1) * (10/3)
    
    cols_to_agg = ['IP_real', 'H','BFP', 'HR', 'R', 'ER', 'BB', 'IB', 'SO', 'SH', 'SF', 'WP', 'HBP', 'BK',
       '2B', '3B']
    winsizes = [10,35,75]
    for winsize in winsizes:
        for raw_col in cols_to_agg:
            new_colname = 'rollsum_'+raw_col+'_'+str(winsize)        
            pitch_df[new_colname] = roll_column(pitch_df, raw_col, winsize)

    
    er_per_ip_def = (5/9)
    h_bb_per_ip_def = 1.5
    h_bb_per_bf_def = .37
    so_per_bf_def = .2
    ip_per_game_def = 3
    bf_per_game_def = 12
    tb_bb_perc_def = .45
    fip_numer_per_ip_def = .124*13 + 1.5*3 - 2*.8
    fip_numer_per_bf_def = .03*13 + .37*3 - 2*.2
    for winsize in winsizes:
        hit_col = 'rollsum_H_'+str(winsize)
        bb_col = 'rollsum_BB_'+str(winsize)
        h_bb_col = 'H_BB_roll_'+str(winsize)
        double_col = 'rollsum_2B_'+str(winsize)
        triple_col = 'rollsum_3B_'+str(winsize)
        hr_col = 'rollsum_HR_'+str(winsize)
        xb_col = 'XB_roll_'+str(winsize)
        tb_col = 'TB_roll_'+str(winsize)
        so_col = 'rollsum_SO_'+str(winsize)
        so_mod_col = 'SO_mod_'+str(winsize)
        ip_col = 'rollsum_IP_real_'+str(winsize)
        ip_mod_col = 'IP_mod_'+str(winsize)
        er_col = 'rollsum_ER_'+str(winsize)
        er_mod_col = 'ER_mod_'+str(winsize)
        bf_col = 'rollsum_BFP_'+str(winsize)
        bf_mod_col = 'BF_mod_'+str(winsize)
        era_col = 'ERA_'+str(winsize)
        fip_col = 'FIP_'+str(winsize)
        fip_perc_col = 'FIP_perc_'+str(winsize)
        fip_numer_col = 'FIP_numer_'+str(winsize)
        fip_numer_mod_col = 'FIP_numer_mod_'+str(winsize)
        fip_numer_mod2_col = 'FIP_numer_mod2_'+str(winsize)
        whip_col = 'WHIP_'+str(winsize)
        so_perc_col = 'SO_perc_'+str(winsize)
        h_bb_perc_col = 'H_BB_perc_'+str(winsize)
        h_bb_mod_col = 'H_BB_mod_'+str(winsize)
        h_bb_mod2_col = 'H_BB_mod2_'+str(winsize)
        tb_bb_mod_col = 'TB_BB_mod_'+str(winsize)
        tb_bb_perc_col = 'TB_BB_perc_'+str(winsize)
        pitch_df[h_bb_col] = pitch_df[hit_col]+pitch_df[bb_col]
        pitch_df[xb_col] = pitch_df[double_col]+2*pitch_df[triple_col]+3*pitch_df[hr_col]
        pitch_df[tb_col] = pitch_df[hit_col]+pitch_df[xb_col]
        pitch_df[ip_mod_col] = np.maximum(pitch_df[ip_col], winsize*ip_per_game_def)
        pitch_df[bf_mod_col] = np.maximum(pitch_df[bf_col], winsize*bf_per_game_def)
        pitch_df[er_mod_col] = pitch_df[er_col] + er_per_ip_def*(pitch_df[ip_mod_col]-pitch_df[ip_col])
        pitch_df[fip_numer_col] = 13*pitch_df[hr_col] + 3*pitch_df[h_bb_col] -2*pitch_df[so_col]
        pitch_df[fip_numer_mod_col] = pitch_df[fip_numer_col] + fip_numer_per_ip_def*(pitch_df[ip_mod_col]-pitch_df[ip_col])
        pitch_df[fip_numer_mod2_col] = pitch_df[fip_numer_col] + fip_numer_per_bf_def*(pitch_df[bf_mod_col]-pitch_df[bf_col])
        pitch_df[h_bb_mod_col] = pitch_df[h_bb_col] + h_bb_per_ip_def*(pitch_df[ip_mod_col]-pitch_df[ip_col])
        pitch_df[h_bb_mod2_col] = pitch_df[h_bb_col] + h_bb_per_bf_def*(pitch_df[bf_mod_col]-pitch_df[bf_col])
        pitch_df[so_mod_col] = pitch_df[so_col] + so_per_bf_def*(pitch_df[bf_mod_col]-pitch_df[bf_col])
        pitch_df[tb_bb_mod_col] = (pitch_df[tb_col] + pitch_df[bb_col])+ tb_bb_perc_def*(pitch_df[bf_mod_col]-pitch_df[bf_col])
        pitch_df[era_col] = (pitch_df[er_mod_col]/pitch_df[ip_mod_col])*9
        pitch_df[fip_col] = (pitch_df[fip_numer_mod_col]/pitch_df[ip_mod_col])
        pitch_df[fip_perc_col] = (pitch_df[fip_numer_mod_col]/pitch_df[bf_mod_col])
        pitch_df[whip_col] = pitch_df[h_bb_mod_col]/pitch_df[ip_mod_col]
        pitch_df[so_perc_col] = pitch_df[so_mod_col]/pitch_df[bf_mod_col]
        pitch_df[tb_bb_perc_col] = pitch_df[tb_bb_mod_col]/pitch_df[bf_mod_col]
        pitch_df[h_bb_perc_col] = pitch_df[h_bb_mod2_col]/pitch_df[bf_mod_col]
    pitch_df['date_dblhead'] = (pitch_df['Date'].astype(str) + pitch_df['dblhead_num'].astype(str)).astype(int)
    pitch_df.set_index('date_dblhead', inplace=True)
    return(pitch_df)

In [ ]:
df_good = load_and_process_pitch_df('goodd001','data/pitcher_data/')

In [ ]:
df_good.head(50)

## Load in our game level data

In [ ]:
df=pd.read_csv('df_bp3.csv')

In [ ]:
start_pitchers_h = df.pitcher_start_id_h.unique()
start_pitchers_v = df.pitcher_start_id_v.unique()
start_pitchers_all = np.union1d(start_pitchers_h, start_pitchers_v)


In [ ]:
pitcher_data_dict = {}
for i, p_id in enumerate(start_pitchers_all):
    if i%100==0:
        print(i)
    pitcher_data_dict[p_id] = load_and_process_pitch_df(p_id,'data/pitcher_data/')

In [ ]:
np.array(pitcher_data_dict['goodd001'].columns)

In [ ]:
raw_cols_to_add = ['GS',  'IP',
       'H', 'BFP', 'HR', 'R', 'ER', 'BB', 'IB', 'SO', 'SH', 'SF', 'WP',
       'HBP', 'BK', '2B', '3B', 'IP_real', 'rollsum_IP_real_10', 'rollsum_H_10',
       'rollsum_BFP_10', 'rollsum_HR_10', 'rollsum_R_10', 'rollsum_ER_10',
       'rollsum_BB_10', 'rollsum_IB_10', 'rollsum_SO_10', 'rollsum_SH_10',
       'rollsum_SF_10', 'rollsum_WP_10', 'rollsum_HBP_10',
       'rollsum_BK_10', 'rollsum_2B_10', 'rollsum_3B_10',
       'rollsum_IP_real_35', 'rollsum_H_35', 'rollsum_BFP_35',
       'rollsum_HR_35', 'rollsum_R_35', 'rollsum_ER_35', 'rollsum_BB_35',
       'rollsum_IB_35', 'rollsum_SO_35', 'rollsum_SH_35', 'rollsum_SF_35',
       'rollsum_WP_35', 'rollsum_HBP_35', 'rollsum_BK_35',
       'rollsum_2B_35', 'rollsum_3B_35', 'rollsum_IP_real_75',
       'rollsum_H_75', 'rollsum_BFP_75', 'rollsum_HR_75', 'rollsum_R_75',
       'rollsum_ER_75', 'rollsum_BB_75', 'rollsum_IB_75', 'rollsum_SO_75',
       'rollsum_SH_75', 'rollsum_SF_75', 'rollsum_WP_75',
       'rollsum_HBP_75', 'rollsum_BK_75', 'rollsum_2B_75',
       'rollsum_3B_75', 'H_BB_roll_10', 'XB_roll_10', 'TB_roll_10',
       'IP_mod_10', 'BF_mod_10', 'ER_mod_10', 'FIP_numer_10',
       'FIP_numer_mod_10', 'FIP_numer_mod2_10', 'H_BB_mod_10',
       'H_BB_mod2_10', 'SO_mod_10', 'TB_BB_mod_10', 'ERA_10', 'FIP_10',
       'FIP_perc_10', 'WHIP_10', 'SO_perc_10', 'TB_BB_perc_10',
       'H_BB_perc_10', 'H_BB_roll_35', 'XB_roll_35', 'TB_roll_35',
       'IP_mod_35', 'BF_mod_35', 'ER_mod_35', 'FIP_numer_35',
       'FIP_numer_mod_35', 'FIP_numer_mod2_35', 'H_BB_mod_35',
       'H_BB_mod2_35', 'SO_mod_35', 'TB_BB_mod_35', 'ERA_35', 'FIP_35',
       'FIP_perc_35', 'WHIP_35', 'SO_perc_35', 'TB_BB_perc_35',
       'H_BB_perc_35', 'H_BB_roll_75', 'XB_roll_75', 'TB_roll_75',
       'IP_mod_75', 'BF_mod_75', 'ER_mod_75', 'FIP_numer_75',
       'FIP_numer_mod_75', 'FIP_numer_mod2_75', 'H_BB_mod_75',
       'H_BB_mod2_75', 'SO_mod_75', 'TB_BB_mod_75', 'ERA_75', 'FIP_75',
       'FIP_perc_75', 'WHIP_75', 'SO_perc_75', 'TB_BB_perc_75',
       'H_BB_perc_75']
cols_to_add = ['Strt_'+col+suff for col in raw_cols_to_add for suff in ['_h','_v']]

col_add_dict = {col:np.zeros(df.shape[0]) for col in cols_to_add}

In [ ]:
for i in range(df.shape[0]):
    row = df.iloc[i,:]
    if i%1000==0:
        print(i)
    sp_id_v = row['pitcher_start_id_v']
    sp_id_h = row['pitcher_start_id_h']
    date_dblhead = row['date_dblhead']
    if sp_id_v in pitcher_data_dict.keys():
        curr_df = pitcher_data_dict[sp_id_v]
        if date_dblhead in curr_df.index:
            for col in raw_cols_to_add:
                col_add_dict['Strt_'+col+'_v'][i] = curr_df.loc[date_dblhead,col]
        else:
            print(f'no match for {sp_id_v} date {date_dblhead}')
    if sp_id_h in pitcher_data_dict.keys():
        curr_df = pitcher_data_dict[sp_id_h]
        if date_dblhead in curr_df.index:
            for col in raw_cols_to_add:
                col_add_dict['Strt_'+col+'_h'][i] = curr_df.loc[date_dblhead,col]
        else:
            print(f'no match for {sp_id_h} date {date_dblhead}')

In [ ]:
for col in cols_to_add:
    df[col] = col_add_dict[col]

In [ ]:
df.sample(5)

In [ ]:
df.shape

In [ ]:
(df.Strt_IP_mod_10_h==0).sum()

In [ ]:
(df.Strt_IP_mod_10_v==0).sum()

In [ ]:
df.loc[df.Strt_IP_mod_10_v==0]

In [ ]:
df.loc[df.Strt_IP_mod_10_h==0]

In [ ]:
df.drop(df.index[df.Strt_IP_mod_10_v==0],inplace=True)
df.drop(df.index[df.Strt_IP_mod_10_h==0],inplace=True)
df.shape

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.to_csv('df_bp5.csv', index=False)